# Big G Express Preliminary Explorations

## Imports

In [119]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Read in

In [120]:
faults = pd.read_csv('../data/J1939Faults.csv')
diag = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv')
makes = pd.read_excel('../data/Vehicle_Make.xlsx')
fault_codes = pd.read_excel('../data/Service Fault Codes_1_0_0_167.xlsx')


C:\Users\jrior\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\jrior\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


## The Basics

### faults at a glance

In [121]:
faults.shape

(1187335, 20)

In [122]:
faults.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000


In [123]:
for col in ['EventTimeStamp', 'LocationTimeStamp']:
    faults[col] = faults[col].astype('datetime64[ns]')

In [124]:
faults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187335 entries, 0 to 1187334
Data columns (total 20 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   RecordID               1187335 non-null  int64         
 1   ESS_Id                 1187335 non-null  int64         
 2   EventTimeStamp         1187335 non-null  datetime64[ns]
 3   eventDescription       1126490 non-null  object        
 4   actionDescription      0 non-null        float64       
 5   ecuSoftwareVersion     891285 non-null   object        
 6   ecuSerialNumber        844318 non-null   object        
 7   ecuModel               1122577 non-null  object        
 8   ecuMake                1122577 non-null  object        
 9   ecuSource              1187335 non-null  int64         
 10  spn                    1187335 non-null  int64         
 11  fmi                    1187335 non-null  int64         
 12  active                 11873

### diag at a glance

In [125]:
diag.shape

(12821626, 4)

In [126]:
diag.tail()

,Id,Name,Value,FaultId
12821621,12864020,EngineCoolantTemperature,181.4,1248457
12821622,12864021,ParkingBrake,False,1248457
12821623,12864022,SwitchedBatteryVoltage,14.1,1248457
12821624,12864023,DistanceLtd,28606.65625,1248457
12821625,12864024,LampStatus,1023,1248458


In [127]:
diag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12821626 entries, 0 to 12821625
Data columns (total 4 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   Id       int64 
 1   Name     object
 2   Value    object
 3   FaultId  int64 
dtypes: int64(2), object(2)
memory usage: 391.3+ MB


In [128]:
diag['FaultId'].nunique()

1187335

In [129]:
diag.loc[diag['Value'].str.contains(',')]

,Id,Name,Value,FaultId
200754,201459,EngineOilPressure,"75,98",18402
200755,201460,EngineOilTemperature,"177,575",18402
200757,201462,TurboBoostPressure,"33,35",18402
200760,201465,FuelLtd,"17867,93716715",18402
200761,201466,FuelRate,"20,68474",18402
...,...,...,...,...
5632164,5640222,IntakeManifoldTemperature,"98,6",545727
5632166,5640224,EngineRpm,"1261,5",545727
5632167,5640225,BarometricPressure,"14,2825",545727
5632169,5640227,Speed,"64,47697",545727


In [130]:
diag.loc[diag['Value'].str.contains(','), 'Name'].unique()

array(['EngineOilPressure', 'EngineOilTemperature', 'TurboBoostPressure',
       'FuelLtd', 'FuelRate', 'IntakeManifoldTemperature', 'EngineRpm',
       'BarometricPressure', 'FuelLevel', 'Speed', 'EngineTimeLtd',
       'EngineCoolantTemperature', 'SwitchedBatteryVoltage',
       'DistanceLtd', 'FuelTemperature', 'Throttle', 'AcceleratorPedal'],
      dtype=object)

In [131]:
diag['Value'] = diag['Value'].str.replace(',', '.')

## fault_codes at a glance

In [140]:
fault_codes.shape

(7124, 14)

In [141]:
fault_codes

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,Mechanical failure in the engine timing actuat...
2,Y,113,167,Not Mapped,20,128,3,635,3,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,High signal voltage detected at the engine tim...
3,Y,114,167,Not Mapped,20,128,4,635,4,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,Low voltage detected at the engine timing actu...
4,Y,115,167,190,Not Mapped,Not Mapped,2,612,2,P0008,Red,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...,The ECM has detected that the primary and back...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7119,Y,9996,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7120,Y,9997,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7121,Y,9998,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7122,Y,9999,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN


In [142]:
fault_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7124 entries, 0 to 7123
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Published in CES 14602  7124 non-null   object
 1   Cummins Fault Code      7124 non-null   int64 
 2   Revision                7124 non-null   int64 
 3   PID                     7124 non-null   object
 4   SID                     7124 non-null   object
 5   MID                     7124 non-null   object
 6   J1587 FMI               7124 non-null   int64 
 7   SPN                     7124 non-null   int64 
 8   J1939 FMI               7124 non-null   int64 
 9   J2012 Pcode             7124 non-null   object
 10  Lamp Color              7124 non-null   object
 11  Lamp Device             7124 non-null   object
 12  Cummins Description     7124 non-null   object
 13  Algorithm Description   2005 non-null   object
dtypes: int64(5), object(9)
memory usage: 779.3+ KB


In [145]:
fault_codes['Published in CES 14602'].unique()
fault_codes['Revision'].unique()

array([167], dtype=int64)

In [146]:
fault_codes = fault_codes[['Cummins Fault Code', 'SPN', 'J1939 FMI', 'Lamp Color', 'Lamp Device', 'Cummins Description', 'Algorithm Description']]

## Merge faults and diag

In [132]:
diag_wide = diag.pivot(index = ['FaultId'],columns = 'Name', values = 'Value')

In [133]:
diag_wide.head()

Name,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
FaultId,,,,,,,,,,,,,,,,,,,,,
1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,0,...,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN


In [134]:
diag_wide_booleans = ['CruiseControlActive', 'IgnStatus', 'ParkingBrake']
boolean_mapper = {'True':True, 'False':False}

for col in diag_wide_booleans:
    diag_wide[col] = diag_wide[col].map(boolean_mapper)

In [135]:
for col in diag_wide.loc[:,~diag_wide.columns.isin(diag_wide_booleans)].columns:
    diag_wide[col] = diag_wide[col].astype(float)

In [136]:
diag_wide

Name,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
FaultId,,,,,,,,,,,,,,,,,,,,,
1,0.0,14.2100,False,66.48672,423178.70000,100.4,11.0,0.00,96.74375,0.00,...,NaN,False,78.8,1023.0,True,NaN,0.00000,3276.75,NaN,0.00
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279.0,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1279.0,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279.0,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,16639.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1023.0,NaN,NaN,NaN,NaN,NaN,NaN
1248455,100.0,14.5000,True,64.62260,423937.90000,185.0,51.0,37.12,211.49370,1310.25,...,32.0,True,98.6,18431.0,False,NaN,65.01096,NaN,73.2,7.83
1248456,0.0,14.3550,True,66.48672,465925.40000,186.8,62.0,41.18,212.84380,1340.75,...,NaN,True,91.4,17407.0,NaN,NaN,66.57410,NaN,100.0,6.96


In [137]:
diag_wide.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1187335 entries, 1 to 1248458
Data columns (total 24 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   AcceleratorPedal           531889 non-null   float64
 1   BarometricPressure         585976 non-null   float64
 2   CruiseControlActive        574916 non-null   object 
 3   CruiseControlSetSpeed      576458 non-null   float64
 4   DistanceLtd                585819 non-null   float64
 5   EngineCoolantTemperature   586071 non-null   float64
 6   EngineLoad                 585621 non-null   float64
 7   EngineOilPressure          586244 non-null   float64
 8   EngineOilTemperature       583912 non-null   float64
 9   EngineRpm                  586921 non-null   float64
 10  EngineTimeLtd              581366 non-null   float64
 11  FuelLevel                  502795 non-null   float64
 12  FuelLtd                    585195 non-null   float64
 13  FuelRate    

In [138]:
faults_diag_wide = faults.merge(diag_wide.reset_index(), left_on = 'RecordID', right_on='FaultId')

In [139]:
faults_diag_wide.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1187335 entries, 0 to 1187334
Data columns (total 45 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   RecordID                   1187335 non-null  int64         
 1   ESS_Id                     1187335 non-null  int64         
 2   EventTimeStamp             1187335 non-null  datetime64[ns]
 3   eventDescription           1126490 non-null  object        
 4   actionDescription          0 non-null        float64       
 5   ecuSoftwareVersion         891285 non-null   object        
 6   ecuSerialNumber            844318 non-null   object        
 7   ecuModel                   1122577 non-null  object        
 8   ecuMake                    1122577 non-null  object        
 9   ecuSource                  1187335 non-null  int64         
 10  spn                        1187335 non-null  int64         
 11  fmi                        1187335 no

In [155]:
full = faults_diag_wide.merge(fault_codes, left_on = ['spn', 'fmi'], right_on = ['SPN', 'J1939 FMI'])

In [162]:
full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1240711 entries, 0 to 1240710
Data columns (total 52 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   RecordID                   1240711 non-null  int64         
 1   ESS_Id                     1240711 non-null  int64         
 2   EventTimeStamp             1240711 non-null  datetime64[ns]
 3   eventDescription           1093560 non-null  object        
 4   actionDescription          0 non-null        float64       
 5   ecuSoftwareVersion         1146003 non-null  object        
 6   ecuSerialNumber            1135428 non-null  object        
 7   ecuModel                   1213715 non-null  object        
 8   ecuMake                    1213715 non-null  object        
 9   ecuSource                  1240711 non-null  int64         
 10  spn                        1240711 non-null  int64         
 11  fmi                        1240711 no

In [160]:
faults_diag_wide['RecordID'].duplicated().sum()

0

In [161]:
full['RecordID'].duplicated().sum()

586062

In [158]:
full.sort_values(['RecordID','Algorithm Description'])#.drop_duplicates(subset=['RecordID'], keep = 'first')

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,...,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,Cummins Fault Code,SPN,J1939 FMI,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,...,3276.75,NaN,0.00,2448,111,17,Maintenance,Maintenance,Coolant Level - Data Valid But Below Normal Op...,Low engine coolant level detected.
640490,2,990360,2015-02-21 11:34:34,NaN,NaN,unknown,unknown,unknown,unknown,11,...,NaN,NaN,NaN,1116,629,12,Amber,Warning,Engine Control Module Critical Internal Failur...,ECM Internal failure has occurred.
677553,5,990416,2015-02-21 11:39:41,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,...,NaN,NaN,NaN,6517,4364,17,None,None,Aftertreatment SCR Catalyst Conversion Efficie...,NaN
2,6,990431,2015-02-21 11:40:22,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,...,3276.75,NaN,6.67,2448,111,17,Maintenance,Maintenance,Coolant Level - Data Valid But Below Normal Op...,Low engine coolant level detected.
4,7,990439,2015-02-21 11:40:52,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,...,3276.75,NaN,20.59,2448,111,17,Maintenance,Maintenance,Coolant Level - Data Valid But Below Normal Op...,Low engine coolant level detected.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974053,1248454,123904424,2020-03-06 14:00:26,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,...,NaN,NaN,NaN,3497,1761,17,Maintenance,Maintenance,Aftertreatment 1 Diesel Exhaust Fluid Tank Lev...,NaN
789109,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,...,NaN,73.2,7.83,3714,1569,31,Amber,Warning,Engine Protection Torque Derate - Condition Ex...,NaN
968871,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,...,NaN,100.0,6.96,3725,3216,10,Amber,Warning,Aftertreatment 1 Intake NOx Sensor - Abnormal ...,NaN
868482,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,...,14.10,100.0,1.74,197,111,18,Amber,Warning,Coolant Level - Data Valid But Below Normal Op...,Low engine coolant level detected.
